In [1]:
!pip install -q transformers datasets peft accelerate bitsandbytes wandb matplotlib seaborn
!pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 138.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.5/132.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 21.9 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig,
    TrainingArguments, Trainer, DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
import wandb
import os
import json

In [2]:
print(f"GPU Available: {torch.cuda.is_available()}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB" if torch.cuda.is_available() else "No GPU")

GPU Available: True
GPU Name: Tesla T4
GPU Memory: 15.8 GB


In [3]:
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abeshith-dsai21-1002 (abeshith-dsai21-1002-dr-m-g-r-educational-and-research-i) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [7]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [8]:
print(f"Model parameters: {base_model.num_parameters():,}")

Model parameters: 1,543,714,304


In [9]:
# Task 1: Math/Reasoning (lightweight)
math_data = load_dataset("microsoft/orca-math-word-problems-200k", split="train[:500]")

# Task 2: Code generation (lightweight)
code_data = load_dataset("iamtarun/python_code_instructions_18k_alpaca", split="train[:500]")

# Task 3: General Q&A (lightweight)
qa_data = load_dataset("databricks/databricks-dolly-15k", split="train[:500]")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/84.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200035 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/905 [00:00<?, ?B/s]

data/train-00000-of-00001-8b6e212f3e1ece(…):   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18612 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [10]:
def format_math_data(examples):
    texts = []
    for i in range(len(examples["question"])):
        question = examples["question"][i]
        answer = examples["answer"][i]
        text = f"Question: {question}\nAnswer: {answer}"
        texts.append(text)
    return {"text": texts}

def format_code_data(examples):
    texts = []
    for i in range(len(examples["instruction"])):
        instruction = examples["instruction"][i]
        output = examples["output"][i]
        text = f"Instruction: {instruction}\nCode: {output}"
        texts.append(text)
    return {"text": texts}

def format_qa_data(examples):
    texts = []
    for i in range(len(examples["instruction"])):
        instruction = examples["instruction"][i]
        response = examples["response"][i]
        text = f"Question: {instruction}\nAnswer: {response}"
        texts.append(text)
    return {"text": texts}

In [11]:
math_formatted = math_data.map(format_math_data, batched=True, remove_columns=math_data.column_names)
code_formatted = code_data.map(format_code_data, batched=True, remove_columns=code_data.column_names)
qa_formatted = qa_data.map(format_qa_data, batched=True, remove_columns=qa_data.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [12]:
# LoRA config for T4 compatibility (small rank)
lora_config = LoraConfig(
    r=8,  # Small rank for T4
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [13]:
training_args = TrainingArguments(
    output_dir="./multi_lora_adapters",
    per_device_train_batch_size=1,  # Small batch for T4
    gradient_accumulation_steps=4,
    num_train_epochs=1,  # Quick training
    max_steps=50,  # Very limited steps
    learning_rate=1e-4,
    bf16=False,  # Use fp16 for T4
    fp16=True,
    optim="paged_adamw_8bit",
    logging_steps=5,
    save_strategy="no",
    report_to="wandb",
    run_name="multi_lora_training"
)

In [14]:
wandb.init(project="multi_lora", name="math_adapter", reinit=True)

wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


In [15]:
model_math = get_peft_model(base_model, lora_config)
model_math.config.use_cache = False

In [16]:
# Tokenize math data
def tokenize_data(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=512,  # Short for T4
        return_tensors=None
    )

math_tokenized = math_formatted.map(tokenize_data, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [17]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Trainer
trainer_math = Trainer(
    model=model_math,
    args=training_args,
    train_dataset=math_tokenized,
    data_collator=data_collator,
)

In [18]:
print("Starting math adapter training...")
trainer_math.train()

Starting math adapter training...


Step,Training Loss
5,0.663400
10,0.659600
15,0.635700
20,0.570300
25,0.544200
30,0.581300
35,0.733100
40,0.547200
45,0.558300
50,0.504600


TrainOutput(global_step=50, training_loss=0.5997601747512817, metrics={'train_runtime': 74.1802, 'train_samples_per_second': 2.696, 'train_steps_per_second': 0.674, 'total_flos': 386865175680000.0, 'train_loss': 0.5997601747512817, 'epoch': 0.4})

In [19]:
model_math.save_pretrained("./math_lora")

In [20]:
wandb.finish()

# Clear memory
del model_math, trainer_math
torch.cuda.empty_cache()

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,▄▁▅▃▂▇▅█▅▃
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,▆▆▅▃▂▃█▂▃▁
total_flos,386865175680000.0
train/epoch,0.4
train/global_step,50
train/grad_norm,0.33809
train/learning_rate,0.0
train/loss,0.5046


In [21]:
wandb.init(project="multi_lora", name="code_adapter", reinit=True)

In [22]:
model_code = get_peft_model(base_model, lora_config)
model_code.config.use_cache = False

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [23]:
code_tokenized = code_formatted.map(tokenize_data, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [24]:
trainer_code = Trainer(
    model=model_code,
    args=training_args,
    train_dataset=code_tokenized,
    data_collator=data_collator,
)

In [25]:
print("Starting code adapter training...")
trainer_code.train()

Starting code adapter training...


Step,Training Loss
5,0.907800
10,0.751000
15,0.889100
20,1.040300
25,0.919300
30,0.804800
35,0.889800
40,0.888400
45,1.011500
50,0.937600


TrainOutput(global_step=50, training_loss=0.9039629411697387, metrics={'train_runtime': 75.3016, 'train_samples_per_second': 2.656, 'train_steps_per_second': 0.664, 'total_flos': 202902417331200.0, 'train_loss': 0.9039629411697387, 'epoch': 0.4})

In [26]:
model_code.save_pretrained("./code_lora")

In [27]:
wandb.finish()

# Clear memory
del model_code, trainer_code
torch.cuda.empty_cache()

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,▄▁▄▂▆▄█▆▆▇
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,▅▁▄█▅▂▄▄▇▆
total_flos,202902417331200.0
train/epoch,0.4
train/global_step,50
train/grad_norm,0.77265
train/learning_rate,0.0
train/loss,0.9376


In [28]:
wandb.init(project="multi_lora", name="qa_adapter", reinit=True)

In [29]:
model_qa = get_peft_model(base_model, lora_config)
model_qa.config.use_cache = False

In [30]:
qa_tokenized = qa_formatted.map(tokenize_data, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [31]:
trainer_qa = Trainer(
    model=model_qa,
    args=training_args,
    train_dataset=qa_tokenized,
    data_collator=data_collator,
)

In [32]:
print("Starting QA adapter training...")
trainer_qa.train()

Starting QA adapter training...


Step,Training Loss
5,2.318400
10,2.282200
15,2.595100
20,2.215200
25,2.127400
30,2.281300
35,2.207300
40,2.153800
45,2.079200
50,1.971200


TrainOutput(global_step=50, training_loss=2.223113899230957, metrics={'train_runtime': 69.6147, 'train_samples_per_second': 2.873, 'train_steps_per_second': 0.718, 'total_flos': 159093135452160.0, 'train_loss': 2.223113899230957, 'epoch': 0.4})

In [33]:
model_qa.save_pretrained("./qa_lora")

In [34]:
wandb.finish()

# Clear memory
del model_qa, trainer_qa
torch.cuda.empty_cache()

train/epoch,▁▂▃▃▄▅▆▆▇██
train/global_step,▁▂▃▃▄▅▆▆▇██
train/grad_norm,▃█▃▄▂▁▃▃▃▆
train/learning_rate,█▇▆▆▅▄▃▃▂▁
train/loss,▅▄█▄▃▄▄▃▂▁
total_flos,159093135452160.0
train/epoch,0.4
train/global_step,50
train/grad_norm,1.21091
train/learning_rate,0.0
train/loss,1.9712


In [35]:
def test_adapter(adapter_path, test_prompt, adapter_name):
    print(f"\n--- Testing {adapter_name} Adapter ---")
    print(f"Prompt: {test_prompt}")

    # Load adapter
    model_with_adapter = PeftModel.from_pretrained(base_model, adapter_path)
    model_with_adapter.eval()

    # Generate
    inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model_with_adapter.generate(
            **inputs,
            max_new_tokens=100,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Response: {response[len(test_prompt):]}")

    # Unload adapter
    del model_with_adapter
    torch.cuda.empty_cache()

    return response

In [36]:
math_result = test_adapter("./math_lora", "Question: What is 15 + 27?", "Math")
code_result = test_adapter("./code_lora", "Instruction: Write a Python function to calculate factorial", "Code")
qa_result = test_adapter("./qa_lora", "Question: What is artificial intelligence?", "QA")


--- Testing Math Adapter ---
Prompt: Question: What is 15 + 27?
Response:  Let's write this in a Python program to solve the problem.
```python
# Calculate the sum of two numbers
sum = 15 + 27

# Print the result
print(sum)
```
This code will output `42`, which is the sum of `15` and `27`.

--- Testing Code Adapter ---
Prompt: Instruction: Write a Python function to calculate factorial
Response:  of a number.
Code: def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n - 1)
print(factorial(5)) # Output: 120

--- Testing QA Adapter ---
Prompt: Question: What is artificial intelligence?
Response:  Answer: Artificial intelligence (AI) refers to the ability of a computer or machine to perform tasks that would normally require human intelligence, such as perception, learning, reasoning and problem solving. It involves creating intelligent agents or machines that can interact with their environment in ways that are similar to how humans do. AI techniques 

In [37]:
def load_and_test_adapter(adapter_path, prompt):
    # Load specific adapter
    model_with_adapter = PeftModel.from_pretrained(base_model, adapter_path)
    model_with_adapter.eval()

    # Generate response
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    with torch.no_grad():
        outputs = model_with_adapter.generate(
            **inputs,
            max_new_tokens=80,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Clean up
    del model_with_adapter
    torch.cuda.empty_cache()

    return response[len(prompt):]

In [42]:
def smart_multi_lora(question):
    question_lower = question.lower()

    # Enhanced math detection
    math_keywords = ["calculate", "math", "number", "add", "subtract", "multiply",
                    "divide", "×", "+", "-", "÷", "/", "*", "=", "sum", "area", "volume"]

    if any(word in question_lower for word in math_keywords):
        print("🔢 Detected: Math question → Using Math LoRA")
        return load_and_test_adapter("./math_lora", question)

    elif any(word in question_lower for word in ["code", "python", "function", "programming", "write", "def"]):
        print("💻 Detected: Coding question → Using Code LoRA")
        return load_and_test_adapter("./code_lora", question)

    else:
        print("❓ Detected: General question → Using QA LoRA")
        return load_and_test_adapter("./qa_lora", question)

In [43]:
test_questions = [
    "Question: What is 25 × 4?",
    "Instruction: Write a Python function to reverse a string",
    "Question: Explain machine learning in simple terms"
]

print("🎯 Testing Smart Multi-LoRA Routing:")
for question in test_questions:
    print(f"\nInput: {question}")
    response = smart_multi_lora(question)
    print(f"Response: {response}")
    print("-" * 50)

🎯 Testing Smart Multi-LoRA Routing:

Input: Question: What is 25 × 4?
🔢 Detected: Math question → Using Math LoRA
Response:  The answer is:
Answer: To find the product of 25 and 4, you simply multiply these two numbers together.

25 × 4 = 100

So, 25 multiplied by 4 equals 100.
--------------------------------------------------

Input: Instruction: Write a Python function to reverse a string
💻 Detected: Coding question → Using Code LoRA
Response:  without using any built-in functions.
Code: def reverse_string(string):
    reversed_str = ""
    for i in range(len(string)-1, -1, -1):
        reversed_str += string[i]
    return reversed_str

print(reverse_string("Hello World")) # prints "dlroW olleH" 
The code above defines a function called `reverse_string` that takes a string
--------------------------------------------------

Input: Question: Explain machine learning in simple terms
❓ Detected: General question → Using QA LoRA
Response: 
Answer: Machine Learning is a branch of Artific

In [40]:
complex_problem = """
I need to:
1. Calculate the area of a circle with radius 5
2. Write Python code to calculate it
3. Explain what pi represents
"""

print(f"Complex Problem: {complex_problem}")

# Break down and solve step by step
steps = [
    ("Question: Calculate the area of a circle with radius 5", "math"),
    ("Instruction: Write Python code to calculate circle area", "code"),
    ("Question: What is pi and why is it important in mathematics?", "qa")
]

for i, (step_question, adapter_type) in enumerate(steps, 1):
    print(f"\nStep {i} ({adapter_type.upper()}):")
    if adapter_type == "math":
        response = load_and_test_adapter("./math_lora", step_question)
    elif adapter_type == "code":
        response = load_and_test_adapter("./code_lora", step_question)
    else:
        response = load_and_test_adapter("./qa_lora", step_question)

    print(f"Question: {step_question}")
    print(f"Answer: {response[:200]}...")

Complex Problem: 
I need to:
1. Calculate the area of a circle with radius 5
2. Write Python code to calculate it
3. Explain what pi represents


Step 1 (MATH):
Question: Question: Calculate the area of a circle with radius 5
Answer:  cm. Round your answer to one decimal place.
Answer Choices:
A) 28.3
B) 10.4
C) 78.5
D) 39.25
E) 62.8

To calculate the area of a circle, you can use the formula:

Area = π * r^2

where r is the...

Step 2 (CODE):
Question: Instruction: Write Python code to calculate circle area
Answer: 
Code: def calculate_area(radius):
    pi = 3.141592653589793
    return (pi * radius ** 2) 

radius = 5

area = calculate_area(radius)

print("Area of a circle with radius", radius, "is:", area)...

Step 3 (QA):
Question: Question: What is pi and why is it important in mathematics?
Answer:  Answer: Pi (π) is a mathematical constant that represents the ratio of a circle's circumference to its diameter. It is approximately equal to 3.14159, but it is an irrational number, me

In [41]:
test_cases = [
    ("Question: What is 123 + 456?", "math"),
    ("Instruction: Create a function to find prime numbers", "code"),
    ("Question: What is the difference between AI and ML?", "qa")
]

performance_results = []

for question, expected_adapter in test_cases:
    print(f"\nTesting: {question}")

    # Test with correct specialized adapter
    specialized_response = load_and_test_adapter(f"./{expected_adapter}_lora", question)

    print(f"Specialized ({expected_adapter.upper()}) Response: {specialized_response[:100]}...")

    performance_results.append({
        'question_type': expected_adapter,
        'specialized_length': len(specialized_response),
        'question': question[:50] + "..."
    })


Testing: Question: What is 123 + 456?
Specialized (MATH) Response:  The answer is:
Answer: 123 + 456 = 579....

Testing: Instruction: Create a function to find prime numbers
Specialized (CODE) Response:  between two given numbers.
Code: def is_prime(num):
    if num > 1:
        for i in range(2, num):...

Testing: Question: What is the difference between AI and ML?
Specialized (QA) Response:  Answer: Artificial Intelligence (AI) refers to a broad field that includes machine learning, natura...
